In [ ]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import plotly.express as px

Seed = 999
torch.manual_seed(Seed)

## Paramètres

In [ ]:
# Chemin vers les images
dataroot = "img_align_celeba/"

# Batch size (combien d'image par combien)
batch_size = 128

# Taille voulue de l'image
image_size = 64

# Nombre de canaux (1 si greyscale 3 si rgb)
nc = 3

# Taille du vecteur bruit en entrée du générateur
nz = 100

# Taille de la feature map du générateur
ngf = 64

# Taille de la feature map du discriminateur
ndf = 64

# Nombre d'epochs
num_epochs = 10

# Learning rate (pas pour la descente de gradient)
lr = 0.0001

# Hyperparametres pour Adam
beta1 = 0.5
beta2 = 0.999
# Nombre de GPU disponible
ngpu = 1

## Préparation des données

In [ ]:
# Création du dataset
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

# Dataloader pour utiliser le dataset
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=2)

# Choix du hardware pour les calcul (Si possible GPU)
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

#Vérification
real_batch = next(iter(dataloader))
px.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

In [ ]:
# Initialisation des poids
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
# Generateur DCGAN

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # Taille. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # Taille. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # Taille. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # Taille. (nc) x 64 x 64n_epochs
        )

    def forward(self, input):
        return self.main(input)


In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # Taille (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # Taille. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # Taille. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # Taille. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # Taille. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# Création de l'instance du générateur
netG = Generator(ngpu).to(device)

# Initialisation des poids
netG.apply(weights_init)

# Print the model
print(netG)


# Création de l'instance du générateur
netD = Discriminator(ngpu).to(device)


# Initialisation des poids
netD.apply(weights_init)

# Print the model
print(netD)

In [ ]:
# Fonction de perte (ici Binary Cross Entropy car deux classe réel et fake)
criterion = nn.BCELoss()

# Matrice de bruit fixée
fixed_noise = torch.randn(64, nz, 1, 1, device=device)
print(f'Fixed_noise {fixed_noise.shape}')

# Attribution des classe pour fake et réel
real_label = 1.
fake_label = 0.

# Initialisation des optimiseur
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, beta2))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, beta2))


In [ ]:
# Pour garder les informations
img_list = []
G_losses = []
D_losses = []

print("Starting Training Loop...")

for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):

        ############################
        # On entraine D
        ###########################
        ## Avec les données réelles
        netD.zero_grad()
        # On adapte le batch
        real_cpu = data[0].to(device)#->On met les données sur le bon hardware
        b_size = real_cpu.size(0)#->On récupére la taille du batch
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # On récupére la sortie du discriminateur
        output = netD(real_cpu).view(-1)
        # On calcule l'erreur de classification du discriminateur sur les données réelles
        errD_real = criterion(output, label)
        # On calcule la descente de gradient associé
        errD_real.backward()
        D_x = output.mean().item()

        ## Entrainement sur données simulée
        # On génére un batch de données simulées
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # On génére de fausse image depuis le générateur
        fake = netG(noise)
        label.fill_(fake_label)
        # On classifie le batch avec le discriminateur
        output = netD(fake.detach()).view(-1)
        # On calcule la perte du discriminateur
        errD_fake = criterion(output, label)
        # On calcule la descente de gradient associé
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # On calcule la perte de D
        errD = (errD_real + errD_fake)/2
        # On opère la descente de gradient
        optimizerD.step()

        ############################
        # On entraîne G
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # La perte de G est la perte de D si les images fake étaient vrai
        output = netD(fake).view(-1)
        # On calcule perte de G
        errG = criterion(output, label)
        # On calcule la descente de gradient associé
        errG.backward()
        D_G_z2 = output.mean().item()
        # On opère la descente de gradient associé
        optimizerG.step()

        # On print quelques infos
        if i % 500 == 0:
            print(f'[{epoch}/{num_epochs}][{i}/{len(dataloader)}]\tLoss_D: {errD.item()}\tLoss_G: {errG.item()}\tD(x): {D_x}\tD(G(z)): {D_G_z1} / {D_G_z2}')

        # On enregistre les loss
        G_losses.append(errG.item())
        D_losses.append(errD.item())

    #On enregiste un lot d'image par epoch
    with torch.no_grad():
        fake = netG(fixed_noise).detach().cpu()
        img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

In [ ]:
#Pour sauvegarder les images
for img, i in zip(img_list,range(10)):
    fig=px.imshow(np.transpose(img,(1,2,0)))
    fig.write_image(f"DCGAN/{i}.jpeg")